In [31]:
import pylsl
import time
import pandas as pd
import numpy as np
import webbrowser
import collections
from collections import deque

In [32]:
# getting stimuli info
stimuli_info = pd.read_csv('/Users/noorelbanna/Desktop/ProjectMUSEEG/Experiment/StimuliSet/relevant_vidData.csv')
print(stimuli_info.columns.to_list())

['clip_name', 'video_url', 'clip_duration', 'rating']


In [33]:
# lsls markers outlet
def lsl_marker_outlet(name):
    # Make a new marker stream
    info = pylsl.stream_info(name, 'Markers', 1, 0, pylsl.cf_string, 'eeg_markers')
    outlet = pylsl.stream_outlet(info, 1, 1)
    return outlet

In [34]:
print("Looking for an EEG stream from OpenBCI GUI...")
streams = pylsl.resolve_stream('type', 'EEG')
inlet = pylsl.StreamInlet(streams[0])
print("EEG stream found!")

Looking for an EEG stream from OpenBCI GUI...
EEG stream found!


In [35]:
# stream info
for s in streams:
    print("Stream name:", s.name())
    print("Type:", s.type())
    print("Nominal sampling rate:", s.nominal_srate())
    print("UID:", s.uid())
    print("---")

Stream name: muse_egg
Type: EEG
Nominal sampling rate: 125.0
UID: 45c3e33b-8a92-419b-8a87-fb62a07679b6
---


In [36]:
print("Nominal sampling rate:", inlet.info().nominal_srate())
print("Time correction:", inlet.time_correction())


Nominal sampling rate: 125.0
Time correction: -4.944300053466577e-05


In [37]:
# initalize LSL marker streams
marker_outlet = lsl_marker_outlet('EEG_Markers') 

2025-10-14 17:01:57.629 (3786.899s) [           2934E]         udp_server.cpp:82    WARN| Could not bind multicast responder for ff02:113d:6fdd:2c17:a643:ffe2:1bd1:3cd2 to interface ::1 (Can't assign requested address)
2025-10-14 17:01:57.632 (3786.902s) [           2934E]         udp_server.cpp:82    WARN| Could not bind multicast responder for ff02:113d:6fdd:2c17:a643:ffe2:1bd1:3cd2 to interface 2600:1700:5d81:30d0:8cf:6f3:681:7535 (Can't assign requested address)
2025-10-14 17:01:57.632 (3786.902s) [           2934E]         udp_server.cpp:82    WARN| Could not bind multicast responder for ff02:113d:6fdd:2c17:a643:ffe2:1bd1:3cd2 to interface 2600:1700:5d81:30d0:cc13:9198:741:b3a7 (Can't assign requested address)
2025-10-14 17:01:57.632 (3786.902s) [           2934E]         udp_server.cpp:82    WARN| Could not bind multicast responder for ff02:113d:6fdd:2c17:a643:ffe2:1bd1:3cd2 to interface 2600:1700:5d81:30d0::28 (Can't assign requested address)
2025-10-14 17:01:57.634 (3786.904s) 

In [38]:
subject_name = "insert-name-here2"

In [ ]:
# should collect 18,750 samples for 150 seconds at 125 Hz for first video 
data_segment = deque()

for i, row in stimuli_info.iterrows():
    trial = row['rating']
    url = row['video_url']
    duration = row['clip_duration']  # in seconds

    # open video link in browser
    webbrowser.open(url)
    time.sleep(5)  # type: ignore # wait for 5 seconds to ensure the browser opens
    print(f"Starting trial {i+1} with rating {trial} for duration {duration} seconds.")

    # wait for the duration of the video
    marker_outlet.push_sample([f'START_Trial_{i+1}_Rating_{trial}'])
    print(f"Marker sent: START_Trial_{i+1}_Rating_{trial}")
    
    # Collect EEG data during the video playback
    start_time = time.time()

    while (time.time() - start_time) < duration:
        sample, timestamp = inlet.pull_sample(timeout=0.008)
        if sample:
            data_segment.append([timestamp] + sample)
    
    # After video ends, send end marker
    marker_outlet.push_sample([f'END_Trial_{i+1}_Rating_{trial}'])

    # Save the collected EEG data for this segment
    eeg_array = np.array(data_segment)
    print(f"Collected {eeg_array.shape} samples for this video.")

    np.save(f"/Users/noorelbanna/Desktop/ProjectMUSEEG/Experiment/StimuliSet/{trial}EEG/video_{i}_{subject_name}_EEG.npy", eeg_array)

    # ask if want to continue
    cont = input("Do you want to continue to the next video? (y/n): ")
    if cont.lower() != 'y':
        break
    



Starting trial 1 with rating negative for duration 150.48 seconds.
Marker sent: START_Trial_1_Rating_negative
Collected (9280, 17) samples for this video.
